Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# 01. Logging APIs
This notebook showcase various ways to use the Azure Machine Learning service run logging APIs, and view the results in the Azure portal.

## Prerequisites
Make sure you go through the [configuration notebook](../../../configuration.ipynb) first if you haven't. Also make sure you have tqdm and matplotlib installed in the current kernel.

```
(myenv) $ conda install -y tqdm matplotlib
```

## Validate Azure ML SDK installation and get version number for debugging purposes

In [ ]:
from azureml.core import Experiment, Run, Workspace
import azureml.core
import numpy as np
from tqdm import tqdm

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

pi = 3.1415926535

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

## Set experiment
Create a new experiment (or get the one with such name).

In [ ]:
exp = Experiment(workspace=ws, name='azlab-aml-101')

## Log metrics
We will start a run, and use the various logging APIs to record different types of metrics during the run.

In [ ]:
# start logging for the run
logging = exp.start_logging()

### Tag

In [ ]:
logging.tag('Team', 'MTC Seattle')
# Case sensetive
logging.tag('Owner', 'Hyunsuk.shin')
logging.tag("Features","ALL Features")
logging.tag("MinMaxScaler", 'No')

### Log

In [ ]:
logging.log("NumFeatures",10)
logging.log("NumRecord",100000)
logging.log('alpha',0.05)

### Log List

In [ ]:
# Log a list of values. Note this will generate a single-variable line chart.
logging.log_list(name='oddNum', value=[1,3,5,7,9,11])
logging.log_list(name='revOddNum', value=[11,9,7,5,3,1])

### Log Row

In [ ]:
logging.log_row(name='drawZ', x=-1,y=1)
logging.log_row(name='drawZ', x=1,y=1)
logging.log_row(name='drawZ', x=-1,y=-1)
logging.log_row(name='drawZ', x=1,y=-1)

### Log Table

In [ ]:
logging.log_table("drawTable", {"x":[1, 2, 3], "y":[0.6, 0.7, 0.89]})

In [ ]:
# create a dictionary to hold a table of values
sines = {}
sines['angle'] = []
sines['sine'] = []

for i in tqdm(range(-10, 10)):
    angle = i / 2.0 * pi
    
    # log a 2 (or more) values as a metric repeatedly. This will generate a 2-variable line chart if you have 2 numerical columns.
    logging.log_row(name='Cosine Wave', angle=angle, cos=np.cos(angle))
        
    sines['angle'].append(angle)
    sines['sine'].append(np.sin(angle))

# log a dictionary as a table, this will generate a 2-variable chart if you have 2 numerical columns
logging.log_table(name='Sine Wave', value=sines)

### Log List

In [ ]:
fibonacci_values = [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]
scaled_values = (i * pi for i in fibonacci_values)

# Log a list of values. Note this will generate a single-variable line chart.
logging.log_list(name='Fibonacci', value=scaled_values)

for i in tqdm(range(-10, 10)):
    # log a metric value repeatedly, this will generate a single-variable line chart.
    logging.log(name='Sigmoid', value=1 / (1 + np.exp(-i)))

## Log an image
This is how to log a _matplotlib_ pyplot object.

In [ ]:
%matplotlib inline

# Create a plot
import matplotlib.pyplot as plt
angle = np.linspace(-3, 3, 50) * 2
plt.plot(angle,np.tanh(angle), label='tanh')
plt.legend(fontsize=12)
plt.title('Hyperbolic Tangent', fontsize=16)
plt.grid(True)

# Log the plot to the run.  To log an arbitrary image, use the form run.log_image(name, path='./image_path.png')
logging.log_image(name='Hyperbolic Tangent', plot=plt)

In [ ]:
#https://docs.microsoft.com/en-us/azure/machine-learning/service/media/concept-azure-machine-learning-architecture/workflow.png#lightbox
!curl https://docs.microsoft.com/en-us/azure/machine-learning/service/media/concept-azure-machine-learning-architecture/workflow.png --output ./azmlworkflow.png

logging.log_image(name='AML workFlow',path='./azmlworkflow.png')

### Upload File

In [ ]:
%%writefile .\outputs\pi.txt
3.1415926535

In [ ]:
# import pickle

# with open(".\outputs\pi.txt","wb") as f:
#     pickle.dump(str('3.1415926535'),f)

In [ ]:
props = logging.upload_file(name='pi_3.14', path_or_stream='./outputs/pi.txt')
props.serialize()

### Upload Model

In [ ]:
logging.register_model(model_name='Pi_3.14',model_path ='./outputs/pi.txt')

## Complete Logging

In [ ]:
logging.complete()

Even after the run is marked completed, you can still log things.

## Examine the run

Now let's take a look at the run detail page in Azure portal. Make sure you checkout the various charts and plots generated/uploaded.

In [ ]:
logging

You can get all the metrics in that run back.

In [ ]:
logging.get_metrics()

You can also see the files uploaded for this run.

In [ ]:
logging.get_file_names()

You can also download all the files locally.

In [ ]:
import os
os.makedirs('files', exist_ok=True)

for f in run.get_file_names():
    dest = os.path.join('files', f.split('/')[-1])
    print('Downloading file {} to {}...'.format(f, dest))
    run.download_file(f, dest)   